In [ ]:
import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse.linalg import svds

from google.colab import drive
drive.mount('/content/gdrive')

import os
os.chdir('/content/gdrive/MyDrive/650/team_proj')

Mounted at /content/gdrive


In [ ]:
import time
import torch 
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F

In [ ]:
#jester=pd.read_csv('jester.csv')
gt_matrix=pd.read_pickle('gt_matrix.pkl')
train_matrix=pd.read_pickle('masked_for_train.pkl')

In [ ]:
gt_matrix2=gt_matrix.fillna(-30).drop(['test_joke_idx'], axis=1)
zero_one_gt=pd.DataFrame((gt_matrix2.values > 0).astype(int), index=gt_matrix2.index, columns=gt_matrix2.columns)
zero_one_gt

5  7  8  13  15  16  17  18  19  20  ...  141  142  143  144  145  146  \
0      1  0  0   0   1   0   0   0   0   0  ...    0    0    0    0    0    0   
1      0  1  1   1   1   1   1   0   0   0  ...    0    0    0    0    0    0   
2      1  1  0   0   0   1   1   1   1   1  ...    0    0    0    0    0    0   
3      0  0  0   1   0   0   1   0   1   0  ...    0    0    0    0    0    0   
4      0  0  0   0   1   1   1   1   1   0  ...    0    0    0    0    0    0   
...   .. .. ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ...  ...  ...  ...  ...   
44796  0  1  0   0   1   0   1   0   0   0  ...    0    0    0    0    0    0   
44797  0  0  0   1   0   0   0   0   0   0  ...    0    0    0    0    0    0   
44798  0  1  1   1   1   1   1   1   1   0  ...    0    0    0    0    0    0   
44799  0  0  1   0   0   0   1   0   0   0  ...    0    0    0    0    0    0   
44800  0  1  1   1   1   1   1   1   1   0  ...    0    0    0    0    0    0   

       147  148  149  150  
0        0    0    0    0  
1        0    0    0    0  
2        0    0    0    0  
3        0    0    0    0  
4        0    0    0    0  
...    ...  ...  ...  ...  
44796    0    0    0    0  
44797    0    0    0    0  
44798    0    0    0    0  
44799    0    0    0    0  
44800    0    0    0    0  

[44801 rows x 140 columns]

In [ ]:
zero_one_train=pd.DataFrame((train_matrix.values > 0).astype(int), index=train_matrix.index, columns=train_matrix.columns)
zero_one_train

5  7  8  13  15  16  17  18  19  20  ...  141  142  143  144  145  146  \
0      1  0  0   0   1   0   0   0   0   0  ...    0    0    0    0    0    0   
1      0  1  1   1   1   1   1   0   0   0  ...    0    0    0    0    0    0   
2      1  1  0   0   0   1   1   1   1   1  ...    0    0    0    0    0    0   
3      0  0  0   1   0   0   1   0   1   0  ...    0    0    0    0    0    0   
4      0  0  0   0   1   1   1   1   1   0  ...    0    0    0    0    0    0   
...   .. .. ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ...  ...  ...  ...  ...   
44796  0  0  0   0   1   0   1   0   0   0  ...    0    0    0    0    0    0   
44797  0  0  0   1   0   0   0   0   0   0  ...    0    0    0    0    0    0   
44798  0  1  1   1   1   0   1   1   1   0  ...    0    0    0    0    0    0   
44799  0  0  0   0   0   0   1   0   0   0  ...    0    0    0    0    0    0   
44800  0  1  1   1   1   1   1   1   1   0  ...    0    0    0    0    0    0   

       147  148  149  150  
0        0    0    0    0  
1        0    0    0    0  
2        0    0    0    0  
3        0    0    0    0  
4        0    0    0    0  
...    ...  ...  ...  ...  
44796    0    0    0    0  
44797    0    0    0    0  
44798    0    0    0    0  
44799    0    0    0    0  
44800    0    0    0    0  

[44801 rows x 140 columns]

In [ ]:
zero_one_train.shape

(44801, 140)

In [ ]:
n_jokes=140

In [ ]:
import keras
from keras import layers
#model structure reference: https://blog.keras.io/building-autoencoders-in-keras.html

input = keras.Input(shape=(140,))
encode = layers.Dense(32, activation='relu')(input)
decode = layers.Dense(140, activation='sigmoid')(encode)
autoencoder = keras.Model(input, decode)

In [ ]:
encoder = keras.Model(input, encode) #make encoder
encoded_input = keras.Input(shape=(32,))
decoder_layer = autoencoder.layers[-1]
decoder = keras.Model(encoded_input, decoder_layer(encoded_input)) #make decoder

In [ ]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 140)]             0         
                                                                 
 dense (Dense)               (None, 32)                4512      
                                                                 
 dense_1 (Dense)             (None, 140)               4620      
                                                                 
Total params: 9,132
Trainable params: 9,132
Non-trainable params: 0
_________________________________________________________________


In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
x_train=zero_one_train[:-4000]
x_test=zero_one_train[-4000:]
gt_train=zero_one_gt[:-4000]
gt_test=zero_one_gt[-4000:]

In [ ]:
autoencoder.fit(x_train, gt_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test,gt_test))

Epoch 1/30
160/160 [==============================] - 2s 6ms/step - loss: 0.4518 - val_loss: 0.2539
Epoch 2/30
160/160 [==============================] - 1s 5ms/step - loss: 0.2705 - val_loss: 0.2013
Epoch 3/30
160/160 [==============================] - 1s 4ms/step - loss: 0.2320 - val_loss: 0.1738
Epoch 4/30
160/160 [==============================] - 1s 5ms/step - loss: 0.2080 - val_loss: 0.1549
Epoch 5/30
160/160 [==============================] - 1s 4ms/step - loss: 0.1897 - val_loss: 0.1402
Epoch 6/30
160/160 [==============================] - 1s 4ms/step - loss: 0.1747 - val_loss: 0.1287
Epoch 7/30
160/160 [==============================] - 1s 4ms/step - loss: 0.1626 - val_loss: 0.1198
Epoch 8/30
160/160 [==============================] - 1s 4ms/step - loss: 0.1532 - val_loss: 0.1134
Epoch 9/30
160/160 [==============================] - 1s 4ms/step - loss: 0.1460 - val_loss: 0.1084
Epoch 10/30
160/160 [==============================] - 1s 4ms/step - loss: 0.1406 - val_loss: 0.1046

In [ ]:
encoded_imgs = encoder.predict(x_test)
restored = decoder.predict(encoded_imgs) #decoded images = final prediction for test data

125/125 [==============================] - 0s 1ms/step


In [ ]:
restored.shape

(4000, 140)

In [ ]:
restored=pd.DataFrame(restored, columns=x_test.columns, index=x_test.index )

In [ ]:
from sklearn.metrics import ndcg_score
def get_rank(arr):
  return arr.argsort()[::-1].argsort()+1

In [ ]:
x_test.index

RangeIndex(start=40801, stop=44801, step=1)

In [ ]:
ndcg_cum=0
for user_id in x_test.index:
  gt_rank=get_rank(x_test.loc[user_id])
  pred_rank=get_rank(restored.loc[user_id])
  ndcg_cum+=ndcg_score(np.asarray([gt_rank]), np.asarray([pred_rank]), k=10)

In [ ]:
ndcg_total=ndcg_cum/len(x_test)
ndcg_total #for k=5

0.5296730653756404

In [ ]:
ndcg_total=ndcg_cum/len(x_test)
ndcg_total #for k=10

0.5296730653756404

In [ ]:
hit=0
k=10
for user_id in x_test.index:
  top_k_pred=list(restored.loc[user_id].sort_values(ascending=False).index[:k])
  #top_true=zero_one_gt.loc[user_id].sort|_values(ascending=False).index[0] 
  top_true=gt_matrix.loc[user_id,'test_joke_idx']
  if top_true in top_k_pred:
    hit+=1

In [ ]:
hit_ratio=hit/len(x_test)
hit_ratio #for k=5

0.02925

In [ ]:
hit_ratio=hit/len(x_test)
hit_ratio #for k=10

0.274

In [ ]:
encoded_res = encoder.predict(x_train)
train_restored = decoder.predict(encoded_res)

1276/1276 [==============================] - 2s 2ms/step


* Train performance check

In [ ]:
train_restored=pd.DataFrame(train_restored, columns=x_train.columns, index=x_train.index )

In [ ]:
ndcg_cum=0
for user_id in x_train.index:
  gt_rank=get_rank(x_train.loc[user_id])
  pred_rank=get_rank(train_restored.loc[user_id])
  ndcg_cum+=ndcg_score(np.asarray([gt_rank]), np.asarray([pred_rank]), k=10)

In [ ]:
ndcg_total=ndcg_cum/len(x_train)
ndcg_total #for k=5

0.5738225546693904

In [ ]:
ndcg_total=ndcg_cum/len(x_train)
ndcg_total #for k=10

0.553943194365502

In [ ]:
hit=0
k=10
for user_id in x_test.index:
  top_k_pred=list(restored.loc[user_id].sort_values(ascending=False).index[:k])
  #top_true=zero_one_gt.loc[user_id].sort_values(ascending=False).index[0]
  top_true=gt_matrix.loc[user_id,'test_joke_idx']
  if top_true in top_k_pred:
    hit+=1

In [ ]:
hit_ratio=hit/len(x_test)
hit_ratio #for k=5

0.02925

In [ ]:
hit_ratio=hit/len(x_test)
hit_ratio #for k=10

0.274